In [ ]:
def tpmNormalize(counts, gene_lengths):
    gene_lengths = {}
    for gene_id in counts['index']:
        try:
            gene_lengths[gene_id] = patric.loc[patric['PATRIC ID'] == gene_id]['Length'].item()
        except Exception:
            gene_lengths[gene_id] = np.nan

    rpk = counts.iloc[:, 1:].divide(list(gene_lengths.values()), axis=0)
    tpm = rpk.divide(rpk.sum(axis=0).values / 1e6, axis=1)
    tpm.insert(0, 'index', counts['index'])
    return tpm


